# Project Title

NLTK ile Twitter üzerinde duygu analizi 

## Members

Betül Durkaya, durkaya@itu.edu.tr

## Description of the project

Duygu analizi, bir konu hakkındaki düşüncenin olumlu-olumsuz ölçülmesidir. Bir fikrin duygusal olarak hangi sınıfa girdiğini belirlemeyi amaçlar. Bu sınıflar olumlu(positive), olumsuz(negative) ve nötr(neutral) olarak adlandırılabilir. 
Günümüzde internet kullanımının artışıyla sosyal medyayı takip eden kişi sayısı hızla artmıştır, bu kişiler duygu ve düşüncelerini sosyal medya aracılığıyla duyurmaktalar. Bu düşünceleri pozitif, negatif ve nötr olarak sınıflandırmak için duygu analizine ihtiyaç duyulur. Duygu analizi önemlidir, çünkü insanların bir konu hakkında neyi sevip neyi sevmediğini görmemize yardımcı olur. Örneğin, bir marka ya da bir ürün hakkında müşterilerin ne hissettiklerini ölçmede kullanılabilir. Duygu analizi, sosyal medyada ürünlere, hizmet faaliyetlerine, kampanyalara ve şirketlere ilişkin duyguları belirler. Görüşlerin ağırlıklı olarak negatif olduğu bir durumda şirket nedenlerini analiz edip çözüm yolları arayabilir.

NLTK(natural language toolkit), Python'da doğal dil işleme(natural language processing) için kütüphaneler ve programlar içeren bir araçtır. Başlangıçta Steven Bird, Edward Loper ve Ewan Klein tarafından geliştirilmiştir. Tokenization, ayrıştırma(parsing), sınıflandırma(classification), köklendirme(stemming), etiketleme(tagging), anlamsal akıl yürütme(semantic reasoning) için metin işleme(text processing) kütüphaneleri içerir.

### The methods to be used

Proje için Python programlama dili kullanılacak. Twittera erişim için bazı anahtarları(consumer key, consumer secret, access token ve access token secret) elde etmemiz gerekiyor, bunu Twitter API yardımıyla yapacağız. Twitter'a Python'la yazdığımız kodlarla istekte bulunacağız ve bağlantı tweepy kütüphanesi ve erişim anahtarlarımız sayesinde gerçekleşecek. Ek olarak json kütüphanesini kullanacağız, bu yüzden Twitter'dan dönen cevap json formatında olacak. Başlangıçta json datası bir txt dosyasında biriktirilecek, sonrasında datayı bir databasede saklayacağız. Bundan sonra stringleri istenen bileşenlere ayıracağız. Stringlerin birer parçası olan kelimeleri, cümlecikleri ve sembolleri "token" olarak isimlendrilir, ayrıca bu işleme tokenization ya da tokenizing denir. Ardından stemming ve lemmatization işlemlerini yapacağız. Stemming; sözcüğün kökünü bulmak için kelime son eklerini değiştirme ve kaldırma yöntemidir. Lemmatization ise farklılaşmış sözcük biçimlerini bir araya getirme işlemidir. Sonra, verileri NLTK ve TextBlob kullanarak analiz edeceğiz.

### The data

Twitter üzerinden tweetler data olarak kullanılacak ve tweetler json formatında toplanacak.

## Code


In [1]:
import tweepy
import json
from processes import tokenize_process, sample_analysis


# organizing tweet information
class StreamListener(tweepy.StreamListener):
    def on_connect(self):
        print("Tweet streaming begin.")

    def on_error(self, status):
        print('Error Type: ' + status)
        return False

    def on_data(self, raw):
        data = json.loads(raw)
        tweet_id = data['id_str']  # The ID of tweet from Twitter in string format
        time = data['created_at']  # The time of creation of the tweet
        username = data['user']['screen_name']  # The Tweet author's username
        text = data['text']  # The entire body of the Tweet
        fs = open("tweets.txt", "a")

        try:
            # insert tweet data to tweet.txt file if RT is not exist
            if data['text'].find('RT @') is -1:
                fs.write(tweet_id + '\t' + time + '\t' + username + '\n' + text + '\n\n\n')
                print(tweet_id + '\t' + time + '\t' + username + '\n' + text)
                tokenize_process(tweet_id, text)    # tokenize tweets
                sample_analysis(text)   # Analysis example
            fs.close()
        except Exception as e:
            print(e)
        return True


In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from textblob import Word, TextBlob


def lemmatize_process(word):
    w = Word(word)
    w.lemmatize()
    return w


def stem_process(word):
    grouping = PorterStemmer()
    grouping.stem(word)


def tokenize_process(tweet_id, tweet):
    fs = open("tokenize.txt", "a")
    lemmatized = []
    tokens = word_tokenize(tweet)
    for index in range(len(tokens)):
        lemmatized.append(lemmatize_process(tokens[index]))

    fs.write('id: ' + tweet_id + '{ ')
    fs.write(str(lemmatized))
    fs.write(' }\n\n')
    print(lemmatized)
    fs.close()


def sample_analysis(tweet):
    sample = TextBlob(tweet)
    print(sample.sentiment, '\n')


In [3]:
import tweepy
from tweepy import OAuthHandler
import listening

#twitter api keys to authanticate
consumer_key = "dtKTW35ISxLtPDCszilhNk6R2"
consumer_secret = "pDoisuGvzTGYte5rNEtjguiixVyys0IrL98xu5tzxE7aG8KZCJ"

access_token = "773843946-LBFza4JmUOrzrUOkahDdtHXi6AHwFQyjwH6j7Ibg"
access_token_secret = "JU0ExxNVFkQbBzipW0EOL9Lkb6dm3RadGdfoJeaLdOUgm"

#authentication sections
auth = OAuthHandler(consumer_key, consumer_secret)
print("First auth done")
auth.set_access_token(access_token, access_token_secret)
print("2nd auth done")

#tracked word
WORDS = ['iphone']

#from listening library, StreamListener object created
listener = listening.StreamListener(api=tweepy.API(wait_on_rate_limit=True))
#from tweepy library, Stream begins
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS, languages=["en"]) #filtering keywords

First auth done
2nd auth done
Tracking: ['iphone']
Tweet streaming begin.
853675534395609088	Sun Apr 16 18:24:42 +0000 2017	SolisFarid
Apple iPhone 6 Factory Unlocked 16GB Smartphone FAIR CONDITION Giveaway RT &amp; Follow https://t.co/iLLoEr1Guy
['Apple', 'iPhone', '6', 'Factory', 'Unlocked', '16GB', 'Smartphone', 'FAIR', 'CONDITION', 'Giveaway', 'RT', '&', 'amp', ';', 'Follow', 'https', ':', '//t.co/iLLoEr1Guy']
Sentiment(polarity=0.7, subjectivity=0.9) 

853675565102116864	Sun Apr 16 18:24:49 +0000 2017	DesSays_
Which? I have a 5 lo, https://t.co/ZY0J5rfZdR
['Which', '?', 'I', 'have', 'a', '5', 'lo', ',', 'https', ':', '//t.co/ZY0J5rfZdR']
Sentiment(polarity=0.0, subjectivity=0.0) 

853675597708611585	Sun Apr 16 18:24:57 +0000 2017	ayoo_tezz
Check out this iPhone/iPad game I'm playing called Basketball Agent. https://t.co/D1qAQf7zFR https://t.co/S6e8Lfulhi
['Check', 'out', 'this', 'iPhone/iPad', 'game', 'I', "'m", 'playing', 'called', 'Basketball', 'Agent', '.', 'https', ':', '//t.c

KeyboardInterrupt: 